In [1]:
import pandas as pd
import plotly.graph_objects as go

from const import DATA_DIR

# ============================================================
# LOAD DATA
# ============================================================
excel_path = DATA_DIR / "power-plants" / "Global-Nuclear-Power-Tracker-September-2025.xlsx"

try:
    print("Reading Excel (sheet=Data)...")
    df = pd.read_excel(excel_path, sheet_name="Data")
except Exception as e:
    print(f"ERROR: failed reading input data. {e}")
    raise

df.columns = [c.strip() for c in df.columns]

# ============================================================
# STATUS COLUMN
# ============================================================
status_options = [
    "Status", "status", "Operating Status",
    "Reactor Status", "Status_text"
]
STATUS_COL = next((c for c in status_options if c in df.columns), None)

if STATUS_COL is None:
    print("Status column not found.")
    sys.exit(0)

# ============================================================
# DATA CLEANING & AGGREGATION
# ============================================================
df["status_clean"] = df[STATUS_COL].astype(str).str.strip().str.lower()

df.loc[df["status_clean"].str.contains("cancelled"), "status_clean"] = "cancelled"
df.loc[df["status_clean"].str.contains("shelved"),   "status_clean"] = "shelved"

status_counts = (
    df["status_clean"]
    .value_counts()
    .sort_values(ascending=True)
)

labels = [s.title().replace("-", "–") for s in status_counts.index]
values = status_counts.values.tolist()

# ============================================================
# COLORS — FOCUS & CONTEXT (IDENTICI)
# ============================================================
colors = []
for s in status_counts.index:
    if "operat" in s:
        colors.append("#0d5a8a")      # Focus: Operating
    elif "construct" in s and "pre" not in s:
        colors.append("#8caebf")      # Construction (azzurrino)
    else:
        colors.append("#d9d9d9")      # Context

# ============================================================
# BAR CHART
# ============================================================
fig = go.Figure()

fig.add_bar(
    x=values,
    y=labels,
    orientation="h",
    marker=dict(color=colors),
    width=0.7,
    hoverinfo="skip"
)

# ============================================================
# VALUE LABELS (FIXED — NO ERROR)
# ============================================================
max_val = max(values)

for y, v, c in zip(labels, values, colors):
    is_focus = c == "#0d5a8a"
    fig.add_annotation(
        x=v + max_val * 0.01,
        y=y,
        text=str(int(v)),
        showarrow=False,
        xanchor="left",
        yanchor="middle",
        font=dict(
            size=11,
            color="#333",
            family="Arial",
            weight="bold" if is_focus else "normal"
        )
    )

# ============================================================
# TITLES & LAYOUT
# ============================================================
total = int(sum(values))

fig.update_layout(
    title=dict(
        text=(
            "<b>Global Nuclear Reactor Status (2025)</b><br>"
            f"<span style='font-size:13px;color:#666;'>"
            f"Active fleet vs. cancelled, shelved, or historical projects "
            f"(Total: {total} records)"
            f"</span>"
        ),
        x=0.02,
        y=0.95,
        xanchor="left"
    ),
    # width=1000,
    # height=650,
    margin=dict(l=180, r=40, t=100, b=40),
    plot_bgcolor="white",
    paper_bgcolor="white",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="#222"
    ),
    xaxis=dict(
        visible=False,
        fixedrange=True
    ),
    yaxis=dict(
        ticks="",
        showline=False,
        fixedrange=True
    )
)

fig.show()


Reading Excel (sheet=Data)...


In [2]:
from const import VISUALIZATIONS_DIR

fig.write_html(VISUALIZATIONS_DIR / "global-nuclear-reactor-status.html")